Import Libraries

In [1]:
# ensures that matplotlib plots are disdplayed inside the notebook 
%matplotlib inline

# provides access to system-specific parameters and functions
import sys
# helps with file and directory operations, enviornment variables, and process control
import os
# for image loading, processing, and manipulation
import cv2
import shutil


import tensorflow as tf
from tensorflow import keras


import matplotlib # for plotting and visualiztion
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from mpl_toolkits.mplot3d import axes3d # for creating 3D plots
from tqdm import tqdm # library used for displaying progress bars for loops
from sklearn.model_selection import train_test_split


import pathlib

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from random import randint

print("Version information")

print("python: {}".format(sys.version))
print("matplotlib: {}".format(matplotlib.__version__))
print("numpy: {}".format(np.__version__))
print("tensorflow: {}".format(tf.__version__))
print("current working directory:", os.getcwd()) # prints curr working directory
print("list of files:", os.listdir())  # Lists the files in the current directory
print("cv2 version:", cv2.__version__)

# Load image
import imageio

# automatically reloads modules before executing code
%load_ext autoreload
%autoreload 2

Matplotlib is building the font cache; this may take a moment.


Version information
python: 3.10.6 (v3.10.6:9c7b4bd164, Aug  1 2022, 17:13:48) [Clang 13.0.0 (clang-1300.0.29.30)]
matplotlib: 3.9.2
numpy: 1.26.4
tensorflow: 2.17.0
current working directory: /Users/wesleytam/Desktop/study-material/CS 4641/Alzheimer-Detection/notebooks
list of files: ['2_Model_2.ipynb', '1_CNN.ipynb', '3_Model_3.ipynb']
cv2 version: 4.10.0


In [2]:
DATASET_PATH = "../Data/"
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
IMAGE_SIZE = (496, 248)
data_dir = pathlib.Path(DATASET_PATH)


Checking for corrupt images

In [25]:
from PIL import Image
import os

for root, _, files in os.walk(data_dir):
    for file in files:
        if file.lower().endswith(('jpg', 'jpeg', 'png')):
            try:
                img = Image.open(os.path.join(root, file))
                img.load()  # Load image to detect issues
            except Exception as e:
                print(f"Corrupted image detected: {os.path.join(root, file)} - {e}")

directory setup and split data

In [3]:
print(data_dir)
print(data_dir.exists())

train_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    labels = 'inferred',
    validation_split = VALIDATION_SPLIT,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    subset = "training",
    seed = 123
)
val_ds = keras.utils.image_dataset_from_directory(
    data_dir,
    labels = 'inferred',
    validation_split = VALIDATION_SPLIT,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    subset = "validation",
    seed = 123
)

test_ds = val_ds.shard(num_shards=2, index=0)
val_ds = val_ds.shard(num_shards=2, index=1)

# train = 0.8 test = 0.1 val = 0.1
print("data has been split")

../Data
True
Found 86437 files belonging to 4 classes.
Using 69150 files for training.
Found 86437 files belonging to 4 classes.
Using 17287 files for validation.
data has been split


In [ ]:
# define path to data
data_dir = "../data/"
# split data
split_dir = "../data_split" # where split data will go
# create split directory
os.makedirs(split_dir, exist_ok=True)

train_ratio = 0.7
test_ratio = 0.2
val_ratio = 0.1

classes = ['Mild Dementia', 'Moderate Dementia', 'Non Demented', 'Very mild Dementia']

# create subdirectories 
for split in ['train', 'validation', 'test']:
    split_path = os.path.join(split_dir, split)
    os.makedirs(split_path, exist_ok=True)
    for cls in classes:
        cls_path = os.path.join(split_path, cls)
        os.makedirs(cls_path, exist_ok=True)

# to split and copy files
def split_data(class_name, src_dir, train_ratio, test_ratio, val_ratio):
    class_dir = os.path.join(src_dir, class_name)
    images = os.listdir(class_dir)

    # train test validation split
    train_images, test_images =  train_test_split(images, test_size=(test_ratio + val_ratio))
    val_images, test_images = train_test_split(test_images, test_size=(test_ratio / (test_ratio + val_ratio)))

    # copy images to folders
    for image in train_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(split_dir, 'train', class_name, image))

    for image in val_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(split_dir, 'validation', class_name, image))

    for image in test_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(split_dir, 'test', class_name, image))

for cls in classes:
    split_data(cls, data_dir, train_ratio, test_ratio, val_ratio)

print("data has been split")
print("working directory contents:", os.listdir(split_dir))

preprocessing data -- data augmentation & normalization

In [7]:
IMG_SIZE = 176 # resize images to 176 x 176
BRIGHT_RANGE = [0.8, 1.2] # adjust brightness to 80 - 120% of original
ZOOM = [0.99, 1.01] # range of zoom -- 1% in or out
FILL_MODE = "constant"
DATA_FORMAT = "channels_last"

# imagedatagenerators
# rescale: normalizes pixel values by dividing by 255 --> bring values in range [0, 1]
# brightness_range: randomly adjusts brightness within specified range 
# zoom_range: randomly zoomes in/out within specifed range
# horizontal_flip: randomly flips images horizontally for more variety
# want augmentation in training data but not test/validation --> just rescale
train_image_generator = ImageDataGenerator(rescale = 1./255, brightness_range = BRIGHT_RANGE, zoom_range = ZOOM, data_format = DATA_FORMAT, fill_mode = FILL_MODE, horizontal_flip = True)

validation_image_generator = ImageDataGenerator(rescale = 1./255)

test_image_generator = ImageDataGenerator(rescale = 1./255)

# each batch has 32 images
print("train images - 70%")
train_dataset = train_image_generator.flow_from_directory(batch_size = 32, directory = os.path.join(split_dir, 'train'), target_size = (176, 176), shuffle = True)

print("test images - 20%")
test_dataset = test_image_generator.flow_from_directory(batch_size = 32, directory = os.path.join(split_dir, 'test'), target_size = (176, 176), shuffle = False)

print("validation images - 10%")
validation_dataset = validation_image_generator.flow_from_directory(batch_size = 32, directory = os.path.join(split_dir, 'validation'), target_size = (176, 176), shuffle = False)


train images - 70%


FileNotFoundError: [WinError 3] The system cannot find the path specified: '../data_split\\train'

Visualize batch of images from dataset 

In [3]:
def image_sample_visualization(generator, y_pred = None):
    labels = dict(zip([0, 1, 2, 3], classes))

    # get batch of images and labels
    x, y = generator.next()

    # display grid of 9 images
    plt.figure(figsize = (10, 10))
    if y_pred is None:
        # display actually class labels for 9 random iamges
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            index = randint(0, len(x) - 1) #randomly select an index within the batch size
            img = x[index]

            # rescale image for display if needed
            if img.max() <= 1:
                img = (img * 255).astype(np.uint8)

            # show image
            plt.imshow(img)
            plt.axis("off")
            plt.title("Class: {}".format(labels[np.argmax(y[index])]))
    else:
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            img = x[i]

            # rescale image for display if needed
            if img.max() <= 1:
                img = (img * 255).astype(np.uint8)

            plt.imshow(x[i])
            plt.axis("off")
            plt.title("Actual:{} \nPredicted:{}".format(labels[np.argmax(y[i])], labels[y_pred[i]]))

    plt.tight_layout()
    plt.show()

image_sample_visualization(train_dataset)

[autoreload of numpy.core.multiarray failed: Traceback (most recent call last):
  File "c:\Users\wiito\miniconda3\envs\alzheimer-detection-gpu\lib\site-packages\IPython\extensions\autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "c:\Users\wiito\miniconda3\envs\alzheimer-detection-gpu\lib\site-packages\IPython\extensions\autoreload.py", line 455, in superreload
    if not append_obj(module, old_objects, name, obj):
  File "c:\Users\wiito\miniconda3\envs\alzheimer-detection-gpu\lib\site-packages\IPython\extensions\autoreload.py", line 423, in append_obj
    in_module = hasattr(obj, "__module__") and obj.__module__ == module.__name__
  File "c:\Users\wiito\miniconda3\envs\alzheimer-detection-gpu\lib\site-packages\numpy\core\overrides.py", line 2, in __getattr__
    from numpy._core import overrides
  File "c:\Users\wiito\miniconda3\envs\alzheimer-detection-gpu\lib\site-packages\numpy\_core\overrides.py", line 8, in <module>
    from numpy._core._multi

NameError: name 'train_dataset' is not defined